In [ ]:
print("main")

### load Packages

In [1]:
# Standard library imports
import os
import random
import warnings
import logging

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import gdown
import rpy2.robjects as ro
from rpy2.rinterface import RRuntimeWarning
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
from pulp import LpSolverDefault
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
# Custom or external package imports
from ddop2.newsvendor import (
    DecisionTreeWeightedNewsvendor, KNeighborsWeightedNewsvendor, 
    SampleAverageApproximationNewsvendor, DeepLearningNewsvendor, 
    RandomForestWeightedNewsvendor, GaussianWeightedNewsvendor, 
    LinearRegressionNewsvendor
)
from drf import drf
from dddex.levelSetKDEx_univariate import LevelSetKDEx
from dddex.loadData import loadDataYaz
from dddex.crossValidation import QuantileCrossValidation, groupedTimeSeriesSplit
from joblib import Parallel, delayed
import pandas as pd
from threadpoolctl import threadpool_limits  # Importiere threadpool_limits


# Set pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full column width
pd.set_option('display.max_rows', 10)  # Limit the number of displayed rows
pd.set_option('display.width', 1000)  # Set high enough width to show all columns in a line

# Suppress warnings and logging
warnings.filterwarnings("ignore")  # Suppress all Python warnings
rpy2_logger.setLevel(logging.CRITICAL)  # Only show critical messages from R

# Set R options to suppress warnings and messages
ro.r('while (sink.number() > 0) sink(NULL)')  # Close open sinks to avoid "sink stack full" errors
ro.r('options(warn=-1)')  # Disable all warnings in R
ro.r('suppressMessages(suppressWarnings(library("drf")))')  # Suppress R package messages and warnings

# Set environment variables for R libraries
os.environ['R_LIBS_USER'] = '/usr/lib/R/site-library'
os.environ['R_HOME'] = '/usr/lib/R'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Set random seeds
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
tf.get_logger().setLevel(logging.ERROR)

# Deactivate CBC Solver output
LpSolverDefault.msg = False  # Deactivates the CBC Solver output

# Verify that the current working directory has changed
print("Current working directory:", os.getcwd())

2024-10-18 17:59:34.892978: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 17:59:34.896003: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-18 17:59:34.959314: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-18 17:59:34.964882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-18 17:59:36.014734: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

Current working directory: /root/WorkingFolder


In [2]:
from scripts.get_data import get_dataset_settings, preprocess_data

from Wrapper.wrapper import DRFWrapper, MLPRegressorWrapper, LevelSetKDExWrapper

from scripts.cv_and_evaluation import pinball_loss, pinball_loss_scorer, preprocess_per_instance, train_and_evaluate_model, calculate_n_iter, bayesian_search_model, preprocess_per_instance, append_result, evaluate_and_append_models, create_cv_folds

from scripts.process_target import process_column

from scripts.get_grids import get_grid

### get data

set configurations in config.py file before we start process

--> set dataset_name before in config file

--> set levelset_calcuations to False if we do the basic models calcuations

In [3]:
import scripts.config as config

dataset_name = config.dataset_name

# Hole die Datei-ID für den gewählten Datensatz
file_id = {
    'bakery': '1r_bDn9Z3Q_XgeTTkJL7352nUG3jkUM0z',
    'yaz': '1xrY3Uv5F9F9ofgSM7dVoSK4bE0gPMg36',
    'm5': '1tCBaxOgE5HHllvLVeRC18zvALBz6B-6w',
    'sid': '1J9bPCfeLDH-mbSnvTHRoCva7pl6cXD3_',
    'air': '1SKPpNxulcusNTjRwCC0p3C_XW7aNBNJZ',
    "copula": '1H5wdJgmxdhbzeS17w0NkRlHRCESEAd-e',
    'wage': '1bn7E7NOoRzE4NwXXs1MYhRSKZHC13qYU',
}[config.dataset_name]


url = f"https://drive.google.com/uc?id={file_id}"


# Datei herunterladen
output = f"{dataset_name}.csv"
gdown.download(url, output, quiet=False)
data = pd.read_csv(output)

# Erstelle die Dataset-Einstellungen basierend auf den geladenen Daten
settings = get_dataset_settings(data)[dataset_name]

y, train_data, test_data, X_train_features, X_test_features, y_train, y_test = preprocess_data(
    data, settings['backscaling_columns'], settings['bool_columns'], settings['drop_columns'])


display(X_train_features.head(10))
display(y_train.head(3))

print("Anzahl der targets:", len(y_train.columns))




Downloading...
From: https://drive.google.com/uc?id=1xrY3Uv5F9F9ofgSM7dVoSK4bE0gPMg36
To: /root/WorkingFolder/yaz.csv
100%|██████████| 3.13M/3.13M [00:00<00:00, 38.9MB/s]


,dayIndex,is_holiday,is_closed,weekend,wind,clouds,rain,sunshine,label,scalingValue,demand__sum_values_7,demand__median_7,demand__mean_7,demand__standard_deviation_7,demand__variance_7,demand__root_mean_square_7,demand__maximum_7,demand__absolute_maximum_7,demand__minimum_7,demand__sum_values_14,demand__median_14,demand__mean_14,demand__standard_deviation_14,demand__variance_14,demand__root_mean_square_14,demand__maximum_14,demand__absolute_maximum_14,demand__minimum_14,demand__sum_values_28,demand__median_28,demand__mean_28,demand__standard_deviation_28,demand__variance_28,demand__root_mean_square_28,demand__maximum_28,demand__absolute_maximum_28,demand__minimum_28,demand,id,weekday_FRI,weekday_MON,weekday_SAT,weekday_SUN,weekday_THU,weekday_TUE,weekday_WED,month_APR,month_AUG,month_DEC,month_FEB,month_JAN,month_JUL,month_JUN,month_MAR,month_MAY,month_NOV,month_OCT,month_SEP,year_2013,year_2014,year_2015
0,29.0,1.0,0.0,0.0,2.6,7.7,0.0,0.0,train,25.0,35.0,5.0,5.000000,1.309307,0.068571,5.168586,8.0,8.0,4.0,77.0,5.0,5.500000,1.762709,0.124286,5.775564,10.0,10.0,3.0,147.0,5.0,5.250000,2.011130,0.161786,5.622023,10.0,10.0,1.0,0.0,calamari,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,30.0,0.0,0.0,1.0,4.4,7.1,0.0,60.0,train,25.0,30.0,4.0,4.285714,2.185294,0.191020,4.810702,8.0,8.0,0.0,70.0,5.0,5.000000,2.203893,0.194286,5.464169,10.0,10.0,0.0,141.0,5.0,5.035714,2.227781,0.198520,5.506490,10.0,10.0,0.0,25.0,calamari,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,31.0,0.0,0.0,1.0,4.8,3.0,0.0,258.0,train,25.0,51.0,5.0,7.285714,7.553888,2.282449,10.494897,25.0,25.0,0.0,85.0,5.0,6.071429,5.522219,1.219796,8.207140,25.0,25.0,0.0,158.0,5.0,5.642857,4.302942,0.740612,7.096277,25.0,25.0,0.0,3.0,calamari,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,32.0,0.0,0.0,0.0,5.1,7.6,0.1,19.0,train,25.0,46.0,4.0,6.571429,7.687785,2.364082,10.113640,25.0,25.0,0.0,82.0,5.0,5.857143,5.578750,1.244898,8.088793,25.0,25.0,0.0,155.0,5.0,5.535714,4.329980,0.749949,7.028005,25.0,25.0,0.0,0.0,calamari,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,33.0,0.0,0.0,0.0,3.5,4.4,0.3,314.0,train,25.0,41.0,4.0,5.857143,8.025470,2.576327,9.935506,25.0,25.0,0.0,77.0,4.5,5.500000,5.778655,1.335714,7.977647,25.0,25.0,0.0,151.0,5.0,5.392857,4.442805,0.789541,6.987233,25.0,25.0,0.0,2.0,calamari,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4427,34.0,0.0,0.0,0.0,2.8,7.8,1.1,0.0,train,82.0,220.0,26.0,31.428571,17.823397,3.874067,36.130715,59.0,59.0,4.0,476.0,29.5,34.000000,14.511080,2.567944,36.967167,59.0,59.0,4.0,891.0,29.0,31.821429,12.920232,2.035761,34.344369,59.0,59.0,4.0,24.0,steak,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4428,35.0,0.0,0.0,0.0,4.0,6.8,0.0,82.0,train,82.0,194.0,24.0,27.714286,16.201537,3.201095,32.102514,59.0,59.0,4.0,472.0,29.5,33.714286,14.664966,2.622698,36.765667,59.0,59.0,4.0,878.0,28.0,31.357143,12.959316,2.048096,33.929549,59.0,59.0,4.0,43.0,steak,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4429,36.0,0.0,0.0,0.0,2.2,7.9,3.8,0.0,train,82.0,197.0,24.0,28.142857,16.556644,3.342957,32.651843,59.0,59.0,4.0,474.0,29.5,33.857143,14.744767,2.651319,36.928502,59.0,59.0,4.0,899.0,29.0,32.107143,13.003679,2.062142,34.640501,59.0,59.0,4.0,23.0,steak,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4430,37.0,0.0,0.0,1.0,3.7,3.2,0.0,151.0,train,82.0,216.0,24.0,30.857143,13.684491,2.283723,33.755423,59.0,59.0,17.0,447.0,28.0,31.928571,14.265164,2.481645,34.970396,59.0,59.0,4.0,885.0,28.0,31.607143,13.074892,2.084790,34.204741,59.0,59.0,4.0,59.0,steak,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


id,calamari,chicken,fish,koefte,lamb,shrimp,steak
dayIndex,,,,,,,
29.0,0.0,1.0,3.0,2.0,0.0,2.0,4.0
30.0,25.0,78.0,14.0,71.0,63.0,23.0,59.0
31.0,3.0,32.0,4.0,19.0,40.0,6.0,26.0


Anzahl der targets: 7


### thread settings 

In [4]:
os.environ['OMP_NUM_THREADS'] = '1'  # OpenMP Threads auf 4 beschränken
os.environ['OPENBLAS_NUM_THREADS'] = '1'  # Für OpenBLAS
os.environ['MKL_NUM_THREADS'] = '1'  # Für Intel MKL (falls verwendet)
os.environ['NUMEXPR_NUM_THREADS'] = '1'  # Für NumExpr
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'  # Für MacOS Accelerate


### process

In [ ]:

# Execution starts here
combinations = [(9, 1), (7.5, 2.5), (5, 5), (2.5, 7.5), (1, 9)]
table_rows = []
random_state = 42

# Initialize cvFolds
cvFolds = None  # Initialization


import scripts.globals as globals  # Import the globals module

for cu, co in combinations:
    print(f"Processing cu, co combination: cu={cu}, co={co}")
    tau = cu / (cu + co)

    # Parallelize column processing within each combination with n_jobs=4 to limit threads
    column_results = Parallel(n_jobs=1)(  
        delayed(process_column)(column, cu, co, tau, y_train, X_train_features, X_test_features, y_test, random_state)
        for column in y_train.columns
    )

    # Combine results from all columns and print after each column
    for result in column_results:
        table_rows.extend(result)
        print(table_rows)
        # Convert the latest result to a DataFrame and print it
        result_table = pd.DataFrame(table_rows, columns=['Variable', 'cu', 'co', 'Model', 'Pinball Loss', 'Best Params', 'delta C', 'sl'])
        print(result_table)  # Print the updated results after each column is processed

# Final result table after processing all combinations
result_table = pd.DataFrame(table_rows, columns=['Variable', 'cu', 'co', 'Model', 'Pinball Loss', 'Best Params', 'delta C', 'sl'])

# Define the folder where results will be saved
results_folder = "results"

# Check if the results folder exists, if not, create it
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Construct the filename using the format "results_basic_Models_{dataset_name}.csv"
filename = os.path.join(results_folder, f"results_basic_Models_{dataset_name}.csv")

# Save the result table to a CSV file in the "results" folder
result_table.to_csv(filename, index=False)

print(f"Results saved as {filename}")

# Aggregate and save cross-validation results at the end of the entire workflow
if globals.global_cv_results:
    # Concatenate all cross-validation results into a single DataFrame
    aggregated_cv_results_df = pd.concat(globals.global_cv_results, ignore_index=True)

    # Print a summary of the aggregated cross-validation data to verify it looks correct
    print("Aggregated cross-validation results sample:")
    print(aggregated_cv_results_df.head(5))  # Print the first 5 rows as a sample

    # Save the aggregated results to a CSV file in the "results" folder
    aggregated_cv_filename = os.path.join(results_folder, f"cv_scores_basic_models_{dataset_name}.csv")
    aggregated_cv_results_df.to_csv(aggregated_cv_filename, index=False)

    print(f"Aggregated cross-validation results saved as {aggregated_cv_filename}")


Processing cu, co combination: cu=9, co=1
Test length for column: 22 1/6 % of: 132
Running model MLP for column calamari, cu=9, co=1
Evaluating model: MLP, cu: 9, co: 1
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Cross-validation results for MLP with cu=9, co=1:
Running model LGBM for column calamari, cu=9, co=1
Evaluating model: LGBM, cu: 9, co: 1
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for ea

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter out both 'SAA' and 'LinearRegression' models
filtered_table = result_table[(~result_table['Model'].isin(['SAA', 'LR', 'MLP','LGBM'])) & (result_table['sl'] == 0.9)]

# Set the figure size
plt.figure(figsize=(10, 6))

# Create the boxplot
sns.boxplot(x='Model', y='delta C', data=filtered_table, showfliers=False, width=0.5, color='lightblue')

 #Add the stripplot to show the individual data points
#sns.stripplot(x='Model', y='delta C', data=filtered_table, color='red', jitter=True, size=6, alpha=0.7)

# Add the point plot to show CI based on SD without horizontal lines
sns.pointplot(x="Model", y="delta C", data=filtered_table, ci='sd', color='blue', markers="o", scale=0.7, linestyles="")

# Add title and labels
plt.title('Boxplot, Stripplot, and Pointplot (CI=SD) of Delta C for Each Model (Excluding SAA and LinearRegression)', fontsize=14)
plt.xlabel('Model', fontsize=12)
plt.ylabel('Delta C', fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

In [10]:
import scripts.config as config

levelset_calculations = True

with open('scripts/config.py', 'w') as f:
    f.write(f"timeseries = {config.timeseries}\n")


In [ ]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
from collections import OrderedDict

timeseries = True

# Execution starts here
combinations = [(9, 1), (7.5, 2.5), (5, 5), (2.5, 7.5), (1, 9)]
table_rows = []
random_state = 1
drf_cv_results = []
global_fold_scores = []

# Iterate over combinations and process them directly
for cu, co in combinations:
    print(f"Processing cu, co combination: cu={cu}, co={co}")
    tau = cu / (cu + co)
    with threadpool_limits(limits=10):
            for column in y_train.columns:
                print(f"Processing column: {column}")

                # Preprocess data
                X_train_scaled, X_test_scaled, y_train_col, y_test_col, X_train_scaled_withID = preprocess_per_instance(
                    column, X_train_features, X_test_features, y_train, y_test
                )
                create_cv_folds(X_train_scaled_withID)
                
                # SAA model evaluation
                saa_model = SampleAverageApproximationNewsvendor(cu, co)
                saa_pred = saa_model.fit(y_train_col).predict(X_test_scaled.shape[0])
                saa_pinball_loss = pinball_loss(y_test_col.values.flatten(), saa_pred, tau)
                append_result(table_rows, column, cu, co, 'SAA', saa_pinball_loss, 'N/A', np.nan, tau)

            
            
                if timeseries:
                    # Initialisiere LGBM und MLP-Modelle
                    lgbm_model = LGBMRegressor(random_state=random_state, n_jobs=1, verbosity=-1)
                    mlp_model = MLPRegressorWrapper(random_state=random_state, early_stopping=True)

                    # LGBM-Modell mit GroupSplitting evaluieren
                    lgbm_model_params = get_grid('LevelSetKDEx_groupsplit', X_train_scaled.shape[1])
                    lgbm_model_evaluation = [
                        ('LS_KDEx_LGBM', LevelSetKDEx(estimator=lgbm_model, binSize=100, weightsByDistance=False), lgbm_model_params)
                    ]
                    evaluate_and_append_models(lgbm_model_evaluation, X_train_scaled, X_test_scaled, y_train_col, y_test_col, saa_pinball_loss, tau, cu, co, column, table_rows, timeseries)

                    # MLP-Modell mit GroupSplitting evaluieren
                    mlp_model_params = get_grid('LevelSetKDEx_groupsplit', X_train_scaled.shape[1])
                    mlp_model_evaluation = [
                        ('LS_KDEx_MLP', LevelSetKDEx(estimator=mlp_model, binSize=100, weightsByDistance=False), mlp_model_params)
                    ]
                    evaluate_and_append_models(mlp_model_evaluation, X_train_scaled, X_test_scaled, y_train_col, y_test_col, saa_pinball_loss, tau, cu, co, column, table_rows, timeseries)


                else:
                    # Although the Wage data set is not a timeseries, it works similarly well on group timeseries splits. 
                    # To do this, we set shuffle = True beforehand in the preprocessing for train/test split. 
                    # Resulting in an mixed order of the train/test points, even if we order it by the "Dayindex" later in the CV splits.
                    # saves the extra work and we can work the same split logic for all datasets
                    print("Only time series Data")

                # DRF-Modell wird immer ausgeführt
                drf_model = DRFWrapper(min_node_size=10, num_trees=100, num_threads=10)
                drf_grid = get_grid('DRF', X_train_scaled.shape[1])
                evaluate_and_append_models([('DRF', drf_model, drf_grid)], X_train_scaled, X_test_scaled, y_train_col, y_test_col, saa_pinball_loss, tau, cu, co, column, table_rows, timeseries)


                # Print the table after evaluating each column
                second_result_table = pd.DataFrame(table_rows, columns=['Variable', 'cu', 'co', 'Model', 'Pinball Loss', 'Best Params', 'delta C', 'sl'])
                print(second_result_table.tail(5))  # Print the last 5 rows of the table after each column is processed


# Define the folder where results will be saved
results_folder = "results"

# Check if the results folder exists, if not, create it
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Final result table after processing all combinations
second_result_table = pd.DataFrame(table_rows, columns=['Variable', 'cu', 'co', 'Model', 'Pinball Loss', 'Best Params', 'delta C', 'sl'])

# Construct the filename and save it in the "results" folder
filename = os.path.join(results_folder, f"results_LevelsetModels_{dataset_name}.csv")
second_result_table.to_csv(filename, index=False)

print(f"Results saved as {filename}")

# Aggregating fold-wise cross-validation results
if globals.global_fold_scores:
    # Reset multi-index for all fold scores
    global_fold_scores_flat = []
    for fold_scores_df in globals.global_fold_scores:
        # Reset the multi-index so that the binSize and weightsByDistance become normal columns
        flat_df = fold_scores_df.reset_index()
        global_fold_scores_flat.append(flat_df)

    # Concatenate all fold-wise cross-validation results into a single DataFrame
    aggregated_fold_scores_df = pd.concat(global_fold_scores_flat, ignore_index=True)


    # Save the aggregated results to a CSV file in the "results" folder
    aggregated_fold_scores_filename = os.path.join(results_folder, f"cv_scores_levelset_models_{dataset_name}.csv")
    aggregated_fold_scores_df.to_csv(aggregated_fold_scores_filename, index=False)

    ### DRF DATA INSERTED INTO THE MAIN TABLE WHERE OTHER BAYES CVs ARE STORED

    aggregated_drf_cv_results_df = pd.concat(globals.drf_cv_results, ignore_index=True)

    # Save the aggregated DRF results to a CSV file in the "results" folder
    aggregated_cv_filename = os.path.join(results_folder, f"cv_drf_scores_{dataset_name}.csv")
    aggregated_drf_cv_results_df.to_csv(aggregated_cv_filename, index=False)
